In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
import pickle 
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
sc = pyspark.SparkContext()
temp_dir = working_dir = os.path.join(os.environ["HAIL_HOME"], "tmp")
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

/opt/sanger.ac.uk/hgi/anaconda3/lib/python3.7/site-packages/hail/context.py:71: UserWarning: pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jar
  'pip-installed Hail requires additional configuration options in Spark referring\n'
Running on Apache Spark version 2.4.3
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.31-6060f9c971cc
LOGGING: writing to /opt/sanger.ac.uk/hgi/hail/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20200727-1455-0.2.31-6060f9c971cc.log


Loading BokehJS ...

In [16]:
#min_prob= 0.5 PC1-PC3
#hdfs://spark-master:9820//ddd-elgh-ukbb/pop_assignments_test_minprob_PC1-PC2-PC3.ht

pca_scores=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_test_minprob_PC1-PC2-PC3.ht")
mt= hl.read_matrix_table(f"{temp_dir}/ddd-elgh-ukbb/Sanger_cohorts_chr1-20-XY_new_cohorts_split_multi.mt")

In [17]:
pca_scores=pca_scores.annotate(cohort=mt.cols()[pca_scores.s].cohort)

In [18]:
pca_scores.show()

2020-07-27 15:05:44 Hail: INFO: Coerced sorted dataset


s,known_pop,pca_scores,pop,prob_African,prob_Any other Asian background,prob_Any other Black background,prob_Any other mixed background,prob_Any other white background,prob_Asian or Asian British,prob_Bangladeshi,prob_Black or Black British,prob_British,prob_Caribbean,prob_Chinese,prob_Do not know,prob_Indian,prob_Irish,prob_Mixed,prob_Other ethnic group,prob_Pakistani,prob_Prefer not to answer,prob_White,prob_White and Asian,prob_White and Black African,prob_White and Black Caribbean,cohort
str,str,array<float64>,str,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str
"""EGAN00001006259""","""""","[2.61e-01,-6.19e-02,-5.76e-02]","""Indian""",0.00e+00,1.00e-02,0.00e+00,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,0.00e+00,5.60e-01,0.00e+00,0.00e+00,0.00e+00,4.00e-01,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006260""","""""","[2.56e-01,-5.81e-02,-1.24e-02]","""Indian""",0.00e+00,8.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,7.70e-01,0.00e+00,0.00e+00,3.00e-02,1.10e-01,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006261""","""""","[3.14e-01,-7.68e-02,-1.01e-03]","""Indian""",0.00e+00,4.00e-02,0.00e+00,1.00e-02,0.00e+00,0.00e+00,1.00e-02,0.00e+00,1.00e-02,3.00e-02,0.00e+00,0.00e+00,7.60e-01,0.00e+00,0.00e+00,2.00e-02,7.00e-02,4.00e-02,0.00e+00,1.00e-02,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006263""","""""","[2.28e-01,-7.27e-02,-7.26e-03]","""Indian""",0.00e+00,3.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,8.20e-01,0.00e+00,0.00e+00,1.00e-02,8.00e-02,0.00e+00,0.00e+00,6.00e-02,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006264""","""""","[1.94e-01,-5.14e-02,3.56e-03]","""Indian""",0.00e+00,0.00e+00,0.00e+00,3.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,0.00e+00,6.80e-01,0.00e+00,0.00e+00,1.00e-02,1.20e-01,1.00e-01,0.00e+00,5.00e-02,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006265""","""""","[-1.63e-02,-4.52e-02,3.14e-02]","""British""",0.00e+00,0.00e+00,0.00e+00,0.00e+00,7.00e-02,0.00e+00,0.00e+00,0.00e+00,8.40e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,6.00e-02,0.00e+00,0.00e+00,0.00e+00,3.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006266""","""""","[-5.97e-02,-4.81e-02,-4.70e-03]","""British""",0.00e+00,0.00e+00,0.00e+00,0.00e+00,3.00e-02,0.00e+00,0.00e+00,0.00e+00,9.70e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006267""","""""","[-7.09e-02,-2.24e-03,3.37e-02]","""British""",0.00e+00,0.00e+00,0.00e+00,0.00e+00,5.00e-02,0.00e+00,0.00e+00,0.00e+00,9.40e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,1.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,"""UK_10K_CHD"""
"""EGAN00001006268""","""""","[-5.51e-02,-2.79e-02,3.42e-02]","""British""",0.00e+00,0.00e+00,0.00e+00,0.00e+00,2.00e-02,0.00e+00,0.00e+00,0.00e+00,9.50e-01,0.00e+00,0.00e+00,0.00e+00,0.00e+00,3.00e-02,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,0.00e+00,"""UK_10K_CHD"""


In [19]:
pca_scores.write(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap_min_prob_PC1-PC2-PC3-annotated.ht", overwrite=True)

2020-07-27 15:06:09 Hail: INFO: Coerced sorted dataset
2020-07-27 15:07:03 Hail: INFO: wrote table with 93674 rows in 512 partitions to /opt/sanger.ac.uk/hgi/hail/tmp/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap_min_prob_PC1-PC2-PC3-annotated.ht


In [20]:
pca_scores=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap_min_prob_PC1-PC2-PC3-annotated.ht")
pca_scores.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'known_pop': str 
    'pca_scores': array<float64> 
    'pop': str 
    'prob_African': float64 
    'prob_Any other Asian background': float64 
    'prob_Any other Black background': float64 
    'prob_Any other mixed background': float64 
    'prob_Any other white background': float64 
    'prob_Asian or Asian British': float64 
    'prob_Bangladeshi': float64 
    'prob_Black or Black British': float64 
    'prob_British': float64 
    'prob_Caribbean': float64 
    'prob_Chinese': float64 
    'prob_Do not know': float64 
    'prob_Indian': float64 
    'prob_Irish': float64 
    'prob_Mixed': float64 
    'prob_Other ethnic group': float64 
    'prob_Pakistani': float64 
    'prob_Prefer not to answer': float64 
    'prob_White': float64 
    'prob_White and Asian': float64 
    'prob_White and Black African': float64 
    'prob_White and Black Ca

In [21]:
p1=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[1], xlabel="PC1", ylabel="PC2", label=pca_scores.cohort)
p2=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[2], xlabel="PC1", ylabel="PC3", label=pca_scores.cohort)


show(bokeh.layouts.gridplot([[p1],[p2]]))

In [22]:
p1=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[1], xlabel="PC1", ylabel="PC2", label=pca_scores.pop)
p2=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[2], xlabel="PC1", ylabel="PC3", label=pca_scores.pop)


show(bokeh.layouts.gridplot([[p1],[p2]]))